## Activate Robot Car Obstacle Avoidance

#### Import Libraries and log info

In [ ]:
#!/usr/bin/python3
#coding=utf8
import sys
sys.path.append('/home/pi/work/notebooks/TurboPi/') # cutom version of TurboPI
import cv2
import time
import math
import signal
import threading
import numpy as np
import pandas as pd
from IPython import display
from PIL import Image
from time import sleep
import sys

import FCSRSDK.yaml_handle as yaml_handle
import FCSRSDK.Camera as Camera
import FCSRSDK.FCSRSonar as Sonar
import FCSRSDK.FCSRBoard as Board
import FCSRSDK.FCSRMecanumChassis as mecanum

if sys.version_info.major == 2:
    print('Please run this program with python3!')
    sys.exit(0)

    
print('''
**********************************************************
******** BUSH FCSR Robot Car Avoidance Test **********
**********************************************************
----------------------------------------------------------
FCSR website:https://chandrunarayan.github.io/fcsr
----------------------------------------------------------
Tips:
 * Press ctrl-c or stop button to exit
----------------------------------------------------------
''')

#### Functions Block

In [ ]:

car = mecanum.MecanumChassis()

def initMove():
    global debug
    car.set_velocity(0,90,0) if not debug else print("debug car.set_velocity(0,90,0)") #reset
    servo_data = yaml_handle.get_yaml_data(yaml_handle.servo_file_path)
    servo1 = servo_data['servo1']
    servo2 = servo_data['servo2']
    Board.setPWMServoPulse(1, servo1, 200)
    Board.setPWMServoPulse(2, servo2, 200)


def reset():
    global debug
    global turn
    global speed
    global forward
    global distance
    global old_speed
    global Threshold
    global stopMotor
    global __isRunning
    
    speed = 40
    old_speed = 0
    distance = 500
    Threshold = 30.0
    turn = True
    forward = True
    stopMotor = True
    __isRunning = False
    

def init():
    print("Avoidance Init")
    initMove()
    reset()
    
__isRunning = False

def start():
    global debug
    global __isRunning
    global stopMotor
    global forward
    global turn
    
    turn = True
    forward = True
    stopMotor = True
    __isRunning = True
    print("Avoidance Start")


def stop():
    global debug
    global __isRunning
    __isRunning = False
    car.set_velocity(0,90,0) if not debug else print("debug car.set_velocity(0,90,0)") #reset
    time.sleep(0.3)
    car.set_velocity(0,90,0) if not debug else print("debug car.set_velocity(0,90,0)") #reset
    print("Avoidance Stop")


def exit():
    global debug
    global __isRunning
    __isRunning = False
    car.set_velocity(0,90,0) if not debug else print("debug car.set_velocity(0,90,0)") #reset
    time.sleep(0.3)
    car.set_velocity(0,90,0) if not debug else print("debug car.set_velocity(0,90,0)") #reset 
    HWSONAR.setPixelColor(0, Board.PixelColor(0, 0, 0))
    HWSONAR.setPixelColor(1, Board.PixelColor(0, 0, 0))
    print("Avoidance Exit")


def setSpeed(args):
    global speed
    speed = int(args[0])
    return (True, ())
 
   
def setThreshold(args):
    global Threshold
    Threshold = args[0]
    return (True, (Threshold,))


def getThreshold(args):
    global Threshold
    return (True, (Threshold,))


def move():
    global debug
    global turn
    global speed
    global forward
    global distance
    global Threshold
    global old_speed
    global stopMotor
    global __isRunning
    while True:
        if __isRunning:   
            if speed != old_speed:   
                old_speed = speed
                car.set_velocity(speed,90,0) if not debug else print(f'debug car.set_velocity({speed},90,0)') #reset 
                
            if distance <= Threshold:   
                if turn: 
                    turn = False
                    forward = True
                    stopMotor = True
                    car.set_velocity(0,90,-0.5) if not debug else print("debug car.set_velocity(0,90,-0.5)") #rteset
                     
                    time.sleep(0.5)
                
            else:
                if forward: 
                    turn = True
                    forward = False
                    stopMotor = True
                    car.set_velocity(speed,90,0) if not debug else print(f'debug car.set_velocity({speed},90,0)') #reset
        else:
            if stopMotor: 
                stopMotor = False
                car.set_velocity(0,90,0) if not debug else print("debug car.set_velocity(0,90,0)") #reset  
            turn = True
            forward = True
            time.sleep(0.03)
 
def add_dist_to_img(img):
    global debug
    global HWSONAR
    global distance
    global distance_data
    TextSize = 12
    TextColor = (0, 255, 255)
    
    dist = HWSONAR.getDistance() / 10.0 

    distance_data.append(dist) 
    data = pd.DataFrame(distance_data)
    data_ = data.copy()
    u = data_.mean()  
    std = data_.std()  

    data_c = data[np.abs(data - u) <= std]
    distance = data_c.mean()[0]

    if len(distance_data) == 5: 
        distance_data.remove(distance_data[0])

    return cv2.putText(img, "Dist:%.1fcm"%distance, (30, 480-30), cv2.FONT_HERSHEY_SIMPLEX, 1.2, TextColor, 2)

def encode_img_to_jpg(frame):
    frame_img = frame.copy() 
    frame_dist = add_dist_to_img(frame_img)  
    frame_resize = cv2.resize(frame_dist, (640, 480)) 
    frame_rotate = cv2.rotate(frame_resize, cv2.ROTATE_90_COUNTERCLOCKWISE)
    #cv2.imshow('frame', frame_resize)_,result = cv2.imencode('.jpg', frame)
    _,frame_jpg = cv2.imencode('.jpg', frame_rotate)
    return frame_jpg

def manual_stop(signum, frame):
    global debug
    global __isRunning
    
    print('Stopping...')
    __isRunning = False
    car.set_velocity(0,90,0) if not debug else print("debug car.set_velocity(0,90,0)") #reset
    print('Stopped')

#### MAIN Block
MAIN Block



In [ ]:
speed = 40
old_speed = 0
distance = 500
Threshold = 30.0
distance_data = []

turn = True
forward = True
HWSONAR = None
stopMotor = True
__isRunning = False

debug = True  # Do not move SnooPi if True
show = False  # Do not show images if False

thread = threading.Thread(target=move)  # Car movement is handled in it's own thread
thread.setDaemon(True)
thread.start()

if __name__ == '__main__':
    print("debugging - SnooPi movement is suppressed") if debug else print("warning - SnooPi might move suddenly!")
    init()
    start()
    HWSONAR = Sonar.Sonar()
    camera = Camera.Camera()
    camera.camera_open(correction=True)
    
    
    while __isRunning:
        signal.signal(signal.SIGINT, manual_stop)
        img = camera.frame
        if img is not None:
            img_jpg = encode_img_to_jpg(img)
            
            if show:
                display_index = display.Image(data=img_jpg)
                display.display(display_index)
                display.clear_output(wait=True)

            key = cv2.waitKey(1)
            if key == 27:
                break
        else:
            time.sleep(0.01)
            
    camera.camera_close()
    cv2.destroyAllWindows()
     